# UN Votes
Author: Cathy 

## Introduction

How do various countries vote in the United Nations General Assembly, how have their voting patterns evolved throughout time, and how similarly or differently do they view certain issues?
Answering these questions (at a high level) is the focus of this analysis. comment for push

### Libraries

We will use the **pandas** and **seaborn** for data wrangling and visualization, and the **DT** package for interactive display of tabular output, and the **unvotes** package for the data.

In [ ]:
# Display plots inline
%matplotlib inline

# Data libraries
import pandas as pd

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting defaults
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 80

# comment for commit

### Data

The data we're using originally come from the **unvotes** R package.
In the chunk below we modify the data by joining the various data frames provided in the package to help you get started with the analysis.

In [ ]:
un_votes = pd.read_csv("data/un_votes.csv")
un_roll_calls = pd.read_csv("data/un_roll_calls.csv")
un_roll_call_issues = pd.read_csv("data/un_roll_call_issues.csv")

In [ ]:
un_votes.loc[un_votes["country"] == "United Kingdom", "country"] = "UK"
un_votes.loc[un_votes["country"] == "United States", "country"] = "US"

unvotes = un_votes.merge(
    un_roll_calls, how = "inner", on = "rcid"
).merge(
    un_roll_call_issues, how = "inner", on = "rcid"
)

In [ ]:
unvotes

## UN voting patterns

Let's create a data visualisation that displays how the voting record of the *UK & NI* changed over time on a variety of issues, and compares it to two other countries: *US* and *Turkey*.

We can easily change which countries are being plotted by changing which countries the code above `filter`s for.
Note that the country name should be spelled and capitalized exactly the same way as it appears in the data.
See the [Appendix](#appendix) for a list of the countries in the data.

In [ ]:
countries = ["UK", "US", "Turkey"]

votes = unvotes.query(
  "country == @countries"   
).assign(
  year = lambda df: pd.DatetimeIndex(df['date']).year,
  vote = lambda df: (df['vote'] == 'yes').astype(int)
).groupby(
  ["country", "year", "issue"],
  as_index = False
).agg(
  {'vote': 'mean'}   
)

In [ ]:
g = sns.FacetGrid(
    votes, col="issue", hue="country", col_wrap = 3, height=4, aspect=1.2
).map(
    sns.regplot, "year", "vote", lowess=True
).add_legend()

plt.show(g)

## References

1.  David Robinson (2017). [unvotes](https://CRAN.R-project.org/package=unvotes): United Nations General Assembly Voting Data. R package version 0.2.0.
2.  Erik Voeten "Data and Analyses of Voting in the UN General Assembly" Routledge Handbook of International Organization, edited by Bob Reinalda (published May 27, 2013).
3.  Much of the analysis has been modeled on the examples presented in the [unvotes package vignette](https://cran.r-project.org/web/packages/unvotes/vignettes/unvotes.html).
